<a href="https://colab.research.google.com/github/sreeman-11021996/Fashion-Recommendation-system/blob/main/train_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.8 MB/s 


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

fashion-product-images-small.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fashion-product-images-small.zip", "r")
zip_ref.extractall("/content")
zip_ref.close()

In [5]:
import tensorflow
from keras.models import Model

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle
import math
import matplotlib.pyplot as plt
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
from utils import *
from dataset import train_val_split,FashionDataset

importing Jupyter notebook from dataset.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
#Loading the labels and removing missing images:
data=pd.read_csv('/content/styles.csv',usecols=["id","gender","masterCategory",
                                                 "subCategory","articleType",
                                                 "baseColour","season","usage"])

In [9]:
# training and validation dataset
dataset = FashionDataset(data,labels=["gender","masterCategory","subCategory"])

[INFO]: Checking if all images are present


44446it [00:03, 13452.06it/s]


[INFO]: Dropping indices: [6697, 16207, 32324, 36399, 40022]


In [10]:
dataset.df.head()

,id,classes
0,15970.jpg,"(Men, Apparel, Topwear)"
1,39386.jpg,"(Men, Apparel, Bottomwear)"
2,59263.jpg,"(Women, Accessories, Watches)"
3,21379.jpg,"(Men, Apparel, Bottomwear)"
4,53759.jpg,"(Men, Apparel, Topwear)"


In [11]:
train_data, val_data = train_val_split(dataset.df)
print(f"[INFO]: Number of training sampels: {len(train_data)}")
print(f"[INFO]: Number of validation sampels: {len(val_data)}")

[INFO]: Number of training sampels: 39996
[INFO]: Number of validation sampels: 4444


In [13]:
batch_size_ = 64

In [60]:
def multi (generator,mlb_,image_df):
  for x,y in generator:
      indices = y.astype(np.int32).tolist()
      y_multi = mlb_.transform(image_df.iloc[indices]['classes'].values.tolist())
      yield x, y_multi

In [61]:
# training and validation data loader
train_generator = dataset.datagen.flow_from_dataframe(
    train_data, 
    batch_size=batch_size_,
    directory='./images/',
    x_col='id',
    y_col='index',
    class_mode='other',
    color_mode="rgb",
    target_size=(224, 224),
    shuffle=True
)
val_generator = dataset.datagen.flow_from_dataframe(
    val_data, 
    batch_size=batch_size_,
    directory='./images/',
    x_col='id',
    y_col='index',
    class_mode='other',
    color_mode="rgb",
    target_size=(224, 224),
    shuffle=True
)

Found 39996 validated image filenames.
Found 4444 validated image filenames.


In [62]:
from sklearn.preprocessing import MultiLabelBinarizer
# Fit a MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(dataset.df['classes'].values.tolist())

MultiLabelBinarizer()

In [63]:
len(mlb.classes_)

56

In [64]:
train_multilabel_generator = multi(train_generator, mlb, train_data)
val_multilabel_generator = multi(val_generator, mlb, val_data)

In [65]:
# Load model architecture
model_file = open("./myModel_arc.json","r")
loaded_json_model = model_file.read()
model_file.close()
loaded_model = model_from_json(loaded_json_model)

In [66]:
model = Model(inputs=loaded_model.input,outputs=loaded_model.layers[len(loaded_model.layers)-2].output)

In [67]:
num_classes = len(mlb.classes_)

In [68]:
final_model = tensorflow.keras.Sequential([
    model,
    tensorflow.keras.layers.Dense(num_classes,activation="sigmoid",name="fc"+str(num_classes))
])

In [69]:
print("train_num - ",math.ceil(train_data.shape[0]/batch_size_))
print("val_num - ",math.ceil(val_data.shape[0]/batch_size_))

train_num -  625
val_num -  70


In [70]:
lr = 0.001
optim = tensorflow.keras.optimizers.RMSprop(learning_rate=lr)
epochs = 1
step_train = math.ceil(train_data.shape[0]/batch_size_)
step_val = math.ceil(val_data.shape[0]/batch_size_)
loss_func = "binary_crossentropy"

In [71]:
final_model.compile(optimizer=optim,loss = loss_func,metrics=["accuracy"])

In [72]:
# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = final_model.fit(train_multilabel_generator,epochs=epochs,validation_data=val_multilabel_generator,
                          callbacks=callbacks_list,steps_per_epoch=step_train,validation_steps=step_val)

488/625 [======================>.......] - ETA: 1:35:10 - loss: 0.1150 - accuracy: 0.4323

In [ ]:
# load the model
new_model = load_model(filepath)

In [ ]:
# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
new_model.fit(train_multilabel_generator,epochs=epochs,validation_data=val_multilabel_generator,
              callbacks=callbacks_list, verbose=0,steps_per_epoch=step_train,validation_steps=step_val)